In [ ]:
import bloqade
import kirin
import bloqade.squin

## Generating circuit kernels

What is `bloqade.squin`. What is `kirin`. Reference for squin statements.

### Option A: Directly building circuits with `squin` dialect

In [ ]:
# Bell state prep.
@bloqade.squin.kernel
def main_GHZ_state():
    qubits = bloqade.squin.qubit(8)
    bloqade.squin.h(qubits[0]) # Apply a gate directly,
    cnot = bloqade.squin.cnot() # or abstractly define the gate first,
    for i in range(7):
        bloqade.squin.apply(cnot, qubits[i], qubits[i+1]) # then apply it.
    bitstrings = bloqade.squin.measure(*qubits)
    return bitstrings
    

In [ ]:
# Incremental
builder_block = kirin.BlockBuilder(dialect=bloqade.squin.kernel)
qubits = builder_block.append(bloqade.squin.qubit.new(2)).result
builder_block.append(bloqade.squin.h(qubits[0]))
builder_block.append(bloqade.squin.cx(qubits[0], qubits[1]))
bitstrings = builder_block.append(bloqade.squin.measure(qubits[0],qubits[1])).result
builder_block.append(kirin.Return(bitstrings))

main_from_builder = builder_block.kernel()

### Option B: Building circuits in Cirq and other SDKs

In [ ]:
import cirq

def ghz_prep()-> cirq.Circuit:
    """
    Builder function that returns a simple 8-qubit
    GHZ state prep using a log-depth circuit
    """
    qubits = cirq.LineQubit.range(8)
    return cirq.Circuit(
        cirq.H(qubits[0]),
        cirq.CX(qubits[0], qubits[1]),
        cirq.CX(qubits[0], qubits[2]),
        cirq.CX(qubits[1], qubits[3]),
        cirq.CX(qubits[0], qubits[4]),
        cirq.CX(qubits[1], qubits[5]),
        cirq.CX(qubits[2], qubits[6]),
        cirq.CX(qubits[3], qubits[7]),
    )
print(ghz_prep())

              ┌──┐   ┌────┐
0: ───H───@────@──────@───────
          │    │      │
1: ───────X────┼@─────┼@──────
               ││     ││
2: ────────────X┼─────┼┼@─────
                │     │││
3: ─────────────X─────┼┼┼@────
                      ││││
4: ───────────────────X┼┼┼────
                       │││
5: ────────────────────X┼┼────
                        ││
6: ─────────────────────X┼────
                         │
7: ──────────────────────X────
              └──┘   └────┘


The cirq circuit can be converted to a bloqade kernel with a transpilation function. The kernel can be considered as a transformation on the register of qubits it is applied to as arguments, with the return being the qubits that still persist.

In [ ]:
# We can convert this circuit to a bloqade kernel
kernel = bloqade.transpile.cirq_to_kernel(ghz_prep())

# Similar methods exist for other circuit libraries and representations:
# kernel = bloqade.transpile.qiskit_to_kernel(qiskit_circuit)
# kernel = bloqade.transpile.pytket_to_kernel(pytket_circuit)
# kernel = bloqade.transpile.qasm2_to_kernel(qasm_string)

# To be fully encapsulated, the kernel should be called within a main() function.
@bloqade.squin.kernel
def main()-> list[int]:
    # Initialize the register
    q = bloqade.squin.qubit.new(8)
    # Apply the unitary to the register
    q = kernel(*q)
    # Measure all qubits in the register
    bits = bloqade.squin.measure(*q)
    return bits

# Alternatively, you can wrap a kernel with a premade decorator
main2 = bloqade.squin.utils.wrap_kernel(kernel) # main2 == main

## Simulation, emulation, and analysis

We can simulate the action of kernels using interpreters. Bloqade supports two kinds of interpreters. Concrete emulators execute the kernel as if it would be done on a real quantum device, returning sets of bitstring measurements or probability distributions. Abstract interpreters do not explicitly execute the kernel, and instead can be seen as a transpilation-- that is, the circuit as a compressed representation of the statevector or probability vector.

In [ ]:
# Concrete emulator that explicitly samples from the probability distribution
hardware_emulator = bloqade.emulators.hardware(seed=42)

# Concrete emulator that returns the probability distribution
software_emulator = bloqade.emulators.software()

# Abstract emulator that returns a Cirq circuit
cirq_emulator = bloqade.emulators.cirq_emulator()

# Eventually, this will include actual quantum hardware
# actual_hardware = bloqade.backends.Gemini(auth=auth)

# Run the emulators
bitstrings:dict[int,int] = hardware_emulator.run(main,shots=1000)

distribution:dict[int,float] = software_emulator.run(main)
statevector = software_emulator.state.quantum # This would be empty, given the kernel includes a measurement
distribution2:dict[int,float] = statevector.probability_distribution()
bitstrings2:dict[int,int] = bloqade.utils.sample_from_distribution(distribution, shots=1000,seed=42)

circuit:cirq.Circuit = cirq_emulator.run(main)
print(bitstrings)
print(circuit)

# Composition of kernels

The powerful thing about bloqade kernels is that they allow all of the typical syntax of for loops, if-else statements, function calls, and other powerful abstractions.

In [ ]:

def trotter_layer(qubits:list[cirq.Qid], dt:float = 0.01, J:float = 1, h:float = 1)-> cirq.Circuit:
    """
    Cirq builder function that returns a circuit of
    a Trotter step of the 1D transverse Ising model
    """
    op_zz = cirq.ZZ**(dt * J)
    op_x = cirq.X**(dt * h)
    circuit = cirq.Circuit()
    for i in range(0,len(qubits)-1,2):
        circuit.append(op_zz.on(qubits[i], qubits[i + 1]))
    for i in range(1,len(qubits)-1,2):
        circuit.append(op_zz.on(qubits[i], qubits[i + 1]))
    for i in range(0, len(qubits)):
        circuit.append(op_x.on(qubits[i]))
    return circuit


In [ ]:
# Option A: write the entire circuit directly, then convert it to a kernel
def trotter_circuit(qubits:list[cirq.Qid], steps:int = 10, dt:float = 0.01, J:float = 1, h:float = 1)-> cirq.Circuit:
    circuit = cirq.Circuit()
    for _ in range(steps):
        circuit += trotter_layer(qubits, dt, J, h)
    return circuit

bloqade_trotter_circuit = bloqade.transpile.cirq_to_kernel(
    trotter_circuit(
        qubits=cirq.LineQubit.range(8),
        steps=10,
        dt=0.01,
        J=1,
        h=1))


In [ ]:
# Option B: use a builder function to create the circuit.
bloqade_trotter_layer = bloqade.transpile.cirq_to_kernel(trotter_layer(qubits=cirq.LineQubit.range(8),dt = 0.01, J = 1, h = 1))

@bloqade.squin.kernel
def trotter_for_loop(N:int, steps:int):
    """
    Main function that runs the Trotter circuit for a given number of steps
    """
    qubits = bloqade.squin.qubit.new(N)
    for _ in range(steps):
        qubits = bloqade_trotter_layer(*qubits)

main_trotter = bloqade.squin.utils.wrap_kernel(trotter_for_loop,kwargs={'N':20, 'steps': 10})

In [ ]:
# Option C: write it all in bloqade directly
@bloqade.squin.kernel
def bloqade_trotter(qubits:list[bloqade.squin.qubit], steps:int, dt:float = 0.01, J:float = 1, h:float = 1):
    """
    Main function that runs the Trotter circuit for a given number of steps
    """
    for _ in range(steps):
        zz = bloqade.squin.op.zzpow(dt * J)
        x = bloqade.squin.op.xpow(dt * h)
        for i in range(0, len(qubits) - 1, 2):
            bloqade.squin.op.apply(zz, qubits[i], qubits[i + 1])
        for i in range(1, len(qubits) - 1, 2):
            bloqade.squin.op.apply(zz, qubits[i], qubits[i + 1])
        for i in range(0, len(qubits)):
            bloqade.squin.op.apply(x, qubits[i])

# How to define the number of qubits that the wrapped kernel will use? Just have it as an argument?
main_trotter2 = bloqade.squin.utils.wrap_kernel(bloqade_trotter, nqubits = 8, kwargs={'steps': 10, 'dt': 0.01, 'J': 1, 'h': 1})

[ Differentiate writing circuits in bloqade kernels (parameterized) vs as circuits (static) ]

# Mid-circuit feed forward

In [ ]:
# 1. Single-bit-decision for T state teleportation in cirq and bloqade.

In [ ]:
# 2. Repeat until success for Z(phi) state teleportation in bloqade. Emphisize that this can't be done in cirq.

In [ ]:
# 3. Linear depth GHZ state preparation

Simulating mid-circuit feed-forward: only certain programs can be expressed in cirq

# Compilation and transformation
Reference the list of squin compiler transformations, and generic kirin transformations.

In [ ]:
main_bell_state_compiled = bloqade.squin.passes.CZGateSet()(main_GHZ_state)
trotter_for_loop_flattened = kirin.passes.Flatten()(trotter_for_loop)


# Combine passes together
custom_compiler_pass = kirin.PassManager(pipeline = [bloqade.squin.passes.CZGateSet(),
                                              kirin.passes.Flatten(),
                                              kirin.passes.Fold(),
                                              ],
                                  fixedpoint=True)

# Or even push it to hardware by lowering to the move dialect.
# Note that the custom_compiler_pass has been reused here.
hardware_compiler_pass = kirin.PassManager(pipeline = [custom_compiler_pass,
                                                       bloqade.compiler.squin_to_move()],
                                  fixedpoint=True)